In [207]:
import csv
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import gaussian_kde

In [208]:
#prefix = input("What is the name of the galaxy?")
#for now,:
#prefix='NGC5128-S1'
prefix='CENA-132302'


#real photometry first
real_file = "%s.phot" % prefix
real_df = pd.read_csv(real_file, delim_whitespace=True, header=None) #create data frame 
real_df = real_df.iloc[:,:37] #select till relevant end of data frame (quality flag of I)

#name columns
columns=['extension','chip','x','y','chi','snr','sharp','round','maj_ax','crowd','type',
           'cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V','inst_vega_V','mag_V','Verr','chi_V','snr_V',
           'sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I','nrm_ct_rt_I','nrm_ct_rt_err_I', 
           'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I','crowd_I','flag_I']

In [209]:
#assign columns and preview
real_df.columns=columns

#fix y column if not on first chip 
real_df['y'] = real_df.apply(lambda x: x.y+2000 if x.extension>1 else x.y, axis=1)


In [210]:
#define V-I
real_df['V-I'] = real_df['inst_vega_v'] - real_df['inst_vega_I']


In [212]:
#establish cuts from below line
#$5<=2.5 && $7*$7<=0.09 && $11<=2 && $20>=5 && $24==0 && $33>=5 && $37==0'' > ${TARG}.phot2")

real_cut = real_df[(real_df['chi'] < 2.5) & (real_df['sharp']*real_df['sharp'] <= 0.09) & (real_df['type'] <= 2) 
        & (real_df['snr_V'] >= 5) & (real_df['flag_V'] == 0) & (real_df['snr_I'] >= 5) 
        & (real_df['flag_I'] == 0)]

real_cut[:5]

,extension,chip,x,y,chi,snr,sharp,round,maj_ax,crowd,...,inst_vega_I,mag_I,Ierr,chi_I,snr_I,sharp_I,round_I,crowd_I,flag_I,V-I
29,1,1,1290.38,1609.84,2.37,1044.1,-0.023,0.134,160,0.029,...,18.302,18.293,0.001,2.22,817.6,-0.007,0.142,0.027,0,1.377
50,1,1,1405.26,1550.67,1.66,794.0,-0.013,0.038,125,0.000,...,18.898,18.889,0.002,1.44,626.0,0.006,0.037,0.000,0,1.415
51,1,1,243.82,1223.73,2.11,788.6,0.044,0.034,115,0.000,...,19.246,19.239,0.002,2.02,530.3,0.076,0.054,0.000,0,0.688
54,1,1,611.73,1546.55,2.36,749.5,0.024,0.046,95,0.000,...,19.273,19.265,0.002,2.26,523.6,0.039,0.039,0.000,0,0.832
62,1,1,2412.38,1725.77,2.23,640.1,-0.002,0.033,145,0.000,...,19.663,19.655,0.003,2.14,429.6,0.016,0.034,0.000,0,0.695


In [239]:
#print out real_cut to space separated file
#real_output_file = '%s.phot2' % (prefix)
real_cut_file = real_cut.to_csv('%s.phot2' % (prefix), sep=' ', header = False, index = False)


In [249]:
#fake photometry next
#fake_file='%s.phot.fake' % prefix #select file 

#with open(fake_file) as f:
#    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
#    first_row = next(reader)
#    num_cols = len(first_row) #get number of columns in file

#i = (num_cols - 71) #number of total extra columns 
#num_EF = int(1 + (i/30)) #number of images in each filter
#i_relevant = (num_EF-1)*4 #number of extra columns we care about (relevant)

In [280]:
fake_df = pd.read_csv(fake_file, delim_whitespace=True, header=None) #create data frame 

# calculate number of columns
numberOfColumns = len(fake_df.columns)
numberOfExtraColumns = numberOfColumns - 71
numberOfImagesPerFilter = int(1+(numberOfExtraColumns/30))
numberOfRelevantColumns = (numberOfImagesPerFilter-1)*4

fake_df = fake_df.iloc[:,:(45+(numberOfRelevantColumns))] #select till relevant end of data frame (quality flag of I)

In [281]:
base_columns=['extensionIn','chipIn','xIn','yIn','counts_V1','mag_V1','counts_I1', 'mag_I1','extensionOut','chipOut','xOut','yOut',
           'chi','snr','sharp','round','maj_ax','crowd','type','cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V',
           'inst_vega_V','mag_V','Verr','chi_V','snr_V','sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I',
           'nrm_ct_rt_I','nrm_ct_rt_err_I', 'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I',
           'crowd_I','flag_I'] #base columns

In [282]:
j = 1
k = 1
l = 1

while (j < numberOfImagesPerFilter):
    m = str(j+1)
    base_columns.insert(k+5,'counts_V%s' % m) #note- should check this on with both more and less images...
    base_columns.insert(k+6,'mag_V%s' % m)    
    base_columns.insert(l+9,'counts_I%s' % m) 
    base_columns.insert(l+10,'mag_I%s' % m)    
    j += 1
    k += 2
    l += 4
if j == numberOfImagesPerFilter:
    fake_df.columns=base_columns 

In [283]:
fake_df.columns

Index(['extensionIn', 'chipIn', 'xIn', 'yIn', 'counts_V1', 'mag_V1',
       'counts_V2', 'mag_V2', 'counts_I1', 'mag_I1', 'counts_I2', 'mag_I2',
       'extensionOut', 'chipOut', 'xOut', 'yOut', 'chi', 'snr', 'sharp',
       'round', 'maj_ax', 'crowd', 'type', 'cts_V', 'sky_v', 'nrm_ct_rt_V',
       'nrm_ct_rt_err_V', 'inst_vega_V', 'mag_V', 'Verr', 'chi_V', 'snr_V',
       'sharp_V', 'round_V', 'crowd_V', 'flag_V', 'cts_I', 'sky_I',
       'nrm_ct_rt_I', 'nrm_ct_rt_err_I', 'inst_vega_I', 'mag_I', 'Ierr',
       'chi_I', 'snr_I', 'sharp_I', 'round_I', 'crowd_I', 'flag_I'],
      dtype='object')

In [284]:
# correct Y position if the star is in the upper CCD
fake_df['yIn'] = fake_df.apply(lambda x: x.yIn+2000 if x.extensionIn>1 else x.yIn, axis=1) 

fake_df.iloc[:5,:16] #print some relevant columns to see if correct

,extensionIn,chipIn,xIn,yIn,counts_V1,mag_V1,counts_V2,mag_V2,counts_I1,mag_I1,counts_I2,mag_I2,extensionOut,chipOut,xOut,yOut
0,1,1,23.26,32.83,132473.4,20.099,213755.1,20.099,253001.2,19.093,139746.5,19.093,1,1,23.26,32.83
1,1,1,11.83,59.43,135030.4,20.078,217894.0,20.078,295841.5,18.923,163406.1,18.923,1,1,12.13,59.43
2,1,1,0.78,7.43,138748.3,20.049,223871.7,20.049,350914.4,18.738,193860.9,18.738,1,1,1.03,7.41
3,1,1,1.99,58.86,133915.1,20.087,216094.0,20.087,376595.8,18.661,208023.4,18.661,1,1,1.99,58.85
4,1,1,53.21,57.10,130628.2,20.114,210791.0,20.114,417143.5,18.550,230431.4,18.550,1,1,53.21,57.10


In [285]:
def calculate(input,output):
    sys.stdout.write(str(input[0])+"\n")
    sys.stdout.write(str(output[0])+"\n")

    return -2.5*np.log10(input[0]/output[0])

In [286]:

#fake_df['fake_Vmag'] = (-2.5*np.log10((fake_df['cts_V'])/(fake_df['counts_V1']+fake_df['counts_V2'])))
fake_df['dV'] = np.where(
    (fake_df['chi'] < 2.5) & (fake_df['sharp']*fake_df['sharp'] <= 0.09) & (fake_df['type'] <= 2) 
    & (fake_df['snr_V'] >= 5) & (fake_df['flag_V'] == 0) & (fake_df['snr_I'] >= 5) 
    & (fake_df['flag_I'] == 0),
    calculate(fake_df['cts_V'],(fake_df['counts_V1']+fake_df['counts_V2'])),
    99.999)

fake_df['dI'] = np.where(
    (fake_df['chi'] < 2.5) & (fake_df['sharp']*fake_df['sharp'] <= 0.09) & (fake_df['type'] <= 2) 
    & (fake_df['snr_V'] >= 5) & (fake_df['flag_V'] == 0) & (fake_df['snr_I'] >= 5) 
    & (fake_df['flag_I'] == 0),
    calculate(fake_df['cts_I'],(fake_df['counts_I1']+fake_df['counts_I2'])),
    99.999)



346241.0
346228.5
392490.6
392747.7


In [270]:
#fake_cut = fake_df[(fake_df['chi'] < 2.5) & (fake_df['sharp']*fake_df['sharp'] <= 0.09) & (fake_df['type'] <= 2) 
#        & (fake_df['snr_V'] >= 5) & (fake_df['flag_V'] == 0) & (fake_df['snr_I'] >= 5) 
#        & (fake_df['flag_I'] == 0)]


fake2_df = fake_df[['mag_V1','mag_I1','inst_vega_V','inst_vega_I','dV','dI']].copy()

In [271]:
fake2_df[:5]

,mag_V1,mag_I1,inst_vega_V,inst_vega_I,dV,dI
0,20.099,19.093,20.099,19.094,0.105655,0.156867
1,20.078,18.923,20.183,19.080,99.999000,99.999000
2,20.049,18.738,20.167,18.927,99.999000,99.999000
3,20.087,18.661,20.090,18.659,99.999000,99.999000
4,20.114,18.550,20.111,18.549,0.105655,0.156867


In [ ]:
fake_cut_file = fake_cut.to_csv('%s.phot.fake2' % (prefix), sep=' ', header = False, index = False)